## Objectives
- Create random names for the messages
- Create random message id's for the messages

## 1. Read data and get Kenyan Names 

In [55]:
import pandas as pd
from faker import Faker 
import random
import uuid

In [69]:
df = pd.read_csv('GeneralistRails_Project_MessageData.csv')
df.head(10)

,User ID,Timestamp (UTC),Message Body
0,208,2017-02-01 19:29:05,So it means if u pay ua loan before the due da...
1,208,2017-02-01 19:21:58,The dates of payment are still indicated n no ...
2,208,2017-02-01 19:21:18,Why was my application rejected
3,208,2017-02-01 19:05:45,Hi branch I requested my number to remain the ...
4,218,2017-02-01 16:08:21,I said ill pay 5th esther camoon.. Infact you ...
5,218,2017-02-01 14:07:37,I will pay on sunday of 5th and i will pay al...
6,218,2017-02-01 12:07:07,I have a late source of salary i expected but ...
7,444,2017-02-02 15:57:36,"I will clear my loan before 15nth,kindly bear ..."
8,676,2017-02-03 14:23:45,Hi can i get the batch number
9,676,2017-02-03 14:23:25,Hi can i get the batch number pl


In [71]:
backup = df.copy()

df.shape

(100, 3)

In [75]:
# Soft reset point
df = backup.copy()
df.shape

(100, 3)

Using Faker to fake first names and conjoin them with a list of kenyan ethnic names

In [36]:
kenyanNames = pd.read_json('kenyan_names.json')
# kenyanNames.shape
kenyanNames['names'][1]

'Kiplagat'

In [29]:
fk = Faker()
fk.name().split(" ")[0]

'Jonathan'

## 2. Generating fake names for users

In [50]:
def generateName():
    name = fk.name().split(" ")[0] + ' ' +  kenyanNames['names'][random.randint(0,84)]
    return name

Iterating through the dataset and adding the names. 
We also check if the user id is the same so that one user id has the same name.

In [76]:
names = {}
for index, row in df.iterrows():
        user_id = row['User ID']
        if user_id not in names:
            names[user_id] = generateName()  # Generate a name if not already generated for this user ID
        df.at[index, 'Name'] = names[user_id]

df.sample(10)

,User ID,Timestamp (UTC),Message Body,Name
65,3897,2017-02-03 7:56:41,I've settled many of your loans before please ...,Joel Kiplagat
92,7837,2017-02-01 6:07:30,Why was my loan request rejected and i have be...,Shelby Kimutai
43,2926,2017-02-03 22:42:26,"Hi! I hope this will take you well, I cleared ...",Holly Kipchirchir
26,2035,2017-02-02 17:55:39,Someone used,David Kemei
23,1481,2017-02-03 1:52:01,Hello. Why can't you make the loan payment opt...,Lynn Ouma
97,8392,2017-02-02 13:51:56,I been with u For long an I made amistake but ...,Edward Oduor
64,3897,2017-02-03 8:15:33,I'm expecting to clear by date 8/2/2017,Joel Kiplagat
38,2780,2017-02-01 0:05:55,I cant access your services,Jennifer Ochieng'
98,8647,2017-02-01 15:53:24,"Sorry,I meant December 2016",Michael Ndung'u
56,3112,2017-02-02 11:40:55,"Dis is keynan,can u kindly forward ma details ...",Noah Omondi


## 3. Generating random Message ID's for each message 

In [77]:
messageIDs = []
for index, row in df.iterrows():
    messageID = uuid.uuid4()
    messageIDs.append(messageID)

df['Message ID'] = messageIDs
df.sample(10)

,User ID,Timestamp (UTC),Message Body,Name,Message ID
66,3897,2017-02-03 7:46:47,Hi branch kindly let me sort out the issue in ...,Joel Kiplagat,718421bf-604e-4893-9428-0d7188080379
17,1245,2017-02-03 16:28:35,"Hi, kindly can i have the batch number",Debra Obura,d0a11dee-7036-4d88-94f2-4aa8452ae858
86,6884,2017-02-01 19:40:52,OK I have paid all of it,Jennifer Ogutu,52459c92-e520-4526-a692-4d017406fcda
5,218,2017-02-01 14:07:37,I will pay on sunday of 5th and i will pay al...,Peter Obiero,13b6191b-4684-400b-8f9a-060e6d356d3f
96,8125,2017-02-01 2:20:12,Will pay be4 15th,Jeffrey Omollo,831dfa2a-6dba-4340-b5e5-a4e4493de2f1
95,8101,2017-02-01 9:43:04,I have been trying this app for a long period....,Christopher Ogada,40a3adc4-d925-46fc-813d-59af2b43f09c
26,2035,2017-02-02 17:55:39,Someone used,David Kemei,e3e2be47-5fae-46a1-8439-c0d9250c4c93
1,208,2017-02-01 19:21:58,The dates of payment are still indicated n no ...,Patrick Kiprop,5211322f-ee81-46f9-8f3d-5f54f73d34e1
42,2884,2017-02-01 7:56:47,I hv my transaction messages with me y am i no...,Joshua Ochieng',1411c826-372b-419c-806b-a4b3e61091c2
59,3701,2017-02-01 12:27:31,Do I have any other loan that I didn't pay,Jill Koech,6b9ee248-66c3-45df-9b96-91c07ff82ab4


And we are done, the data is now more presentable on the front end and easier to use. 